In [1]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 11.7 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random
from torchmetrics.classification import Recall, Accuracy, AUROC, Precision

In [3]:
FILL = "FILL"

In [4]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2023-12-20 18:03:32--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z     [  <=>               ] 198.65K   412KB/s    in 0.5s    

2023-12-20 18:03:33 (412 KB/s) - ‘smsspamcollection.zip’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [5]:
!unzip -o smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [6]:
!head -10 SMSSpamCollection

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [7]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
!unzip -o smsspamcollection.zip

--2023-12-20 18:03:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘smsspamcollection.zip.1’

smsspamcollection.z     [   <=>              ] 198.65K   390KB/s    in 0.5s    

2023-12-20 18:03:35 (390 KB/s) - ‘smsspamcollection.zip.1’ saved [203415]

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [8]:
file_path = 'SMSSpamCollection'
df = pd.DataFrame({'label':int(), 'text':str()}, index = [])
with open(file_path) as f:
    for line in f.readlines():
        split = line.split('\t')
        df = pd.concat([
                df,
                pd.DataFrame.from_dict({
                    'label': [1 if split[0] == 'spam' else 0],
                    'text': [split[1]]
                })
            ],
            ignore_index=True
        )
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
text = df.text.values
labels = df.label.values

In [10]:
# Set to the GPT2Tokenizer and set lower case to True
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
# Set the padding to 'left' or 'right'?
# Remember we want to use the last token's embedding to represent the entire sentence
tokenizer.padding_side = 'right'

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [11]:
def print_rand_sentence():
    '''Displays the tokens and respective IDs of a random text sample'''
    index = random.randint(0, len(text)-1)
    print(text[index])
    table = np.array([tokenizer.tokenize(text[index]),
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

okmail: Dear Dave this is your final notice to collect your 4* Tenerife Holiday or #5000 CASH award! Call 09061743806 from landline. TCs SAE Box326 CW25WX 150ppm

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ ok       │         482 │
├──────────┼─────────────┤
│ mail     │        4529 │
├──────────┼─────────────┤
│ :        │          25 │
├──────────┼─────────────┤
│ ĠDear    │       23420 │
├──────────┼─────────────┤
│ ĠDave    │        9935 │
├──────────┼─────────────┤
│ Ġthis    │         428 │
├──────────┼─────────────┤
│ Ġis      │         318 │
├──────────┼─────────────┤
│ Ġyour    │         534 │
├──────────┼─────────────┤
│ Ġfinal   │        2457 │
├──────────┼─────────────┤
│ Ġnotice  │        4003 │
├──────────┼─────────────┤
│ Ġto      │         284 │
├──────────┼─────────────┤
│ Ġcollect │        2824 │
├──────────┼─────────────┤
│ Ġyour    │         534 │
├──────────┼─────────────┤
│ Ġ4       │         604 │
├──────────┼─────────────┤


In [12]:
import torch
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  # Use the tokenizer and the encode_plus methods to return the right data we'll need
  # Set max_length = 32 and return_tokens = 'pt'
  # Set other fields to the appropriate booleans needed
  return tokenizer.encode_plus(
      input_text,
      add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
      max_length=32,
      truncation=True,
      padding='max_length',  # Pad to max_length
      return_attention_mask=True,
      return_tensors='pt'  # Return PyTorch tensors
    )


for sample in text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids'])
    attention_masks.append(encoding_dict['attention_mask'])


# Gather all the torch_id, attention masks, and labels
token_id = torch.cat(token_id, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [13]:
def print_rand_sentence_encoding():
    '''Displays tokens, token IDs and attention mask of a random text sample'''
    index = random.randint(0, len(text) - 1)
    tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
    print(tokens)
    token_ids = [i.numpy() for i in token_id[index]]
    attention = [i.numpy() for i in attention_masks[index]]
    table = np.array([tokens, token_ids, attention]).T
    print(
        tabulate(
            table,
            headers = ['Tokens', 'Token IDs', 'Attention Mask'],
            tablefmt = 'fancy_grid')
    )

print_rand_sentence_encoding()

['No', 'Ġno', '.', 'ĠI', 'Ġwill', 'Ġcheck', 'Ġall', 'Ġrooms', 'Ġbe', 'for', 'Ġactivities', 'Ċ', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>']
╒═══════════════╤═════════════╤══════════════════╕
│ Tokens        │   Token IDs │   Attention Mask │
╞═══════════════╪═════════════╪══════════════════╡
│ No            │        2949 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġno           │         645 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ .             │          13 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ ĠI            │         314 │                1 │
├───────────────┼─────────────┼──────────────────┤
│ Ġ

In [14]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
# Use train_test_split
from sklearn.model_selection import train_test_split

train_idx, val_idx, _, _ = train_test_split(
    np.arange(len(labels)),  # indices to split
    labels,                  # labels for stratification
    test_size=val_ratio,     # size of validation set
    random_state=42,         # seed for reproducibility
    stratify=labels          # stratify by labels
)

# Train and validation sets
# Set to TensorDataset
train_set = TensorDataset(token_id[train_idx], attention_masks[train_idx], labels[train_idx])
val_set = TensorDataset(token_id[val_idx], attention_masks[val_idx], labels[val_idx])

# Prepare DataLoader
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
    train_set,
    sampler=RandomSampler(train_set),  # Random sampler for training
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    val_set,
    sampler=SequentialSampler(val_set),  # Sequential sampler for validation
    batch_size=batch_size
)

### Load specific versions of the model

In [15]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states
from transformers import GPT2Config, GPT2ForSequenceClassification

config = GPT2Config.from_pretrained('gpt2', output_attentions=False, output_hidden_states=False)

# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, eps=1e-08)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Set the model to the right device

In [16]:

# Check if CUDA is available and set the device to GPU if it is, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

In [18]:
# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = True

# Turn off gradients using the above
total_parameters = 0

for name, param in model.named_parameters():
    param_requires_grad = fine_tune or "score" in name
    param.requires_grad = param_requires_grad

    num_params = param.numel()
    if param_requires_grad:
        total_parameters += num_params

    print(f"{name}: {num_params} parameters, requires_grad={param_requires_grad}")

assert total_parameters == 124441344, f"Total parameters count should be 124441344, but is {total_parameters}"

transformer.wte.weight: 38597376 parameters, requires_grad=True
transformer.wpe.weight: 786432 parameters, requires_grad=True
transformer.h.0.ln_1.weight: 768 parameters, requires_grad=True
transformer.h.0.ln_1.bias: 768 parameters, requires_grad=True
transformer.h.0.attn.c_attn.weight: 1769472 parameters, requires_grad=True
transformer.h.0.attn.c_attn.bias: 2304 parameters, requires_grad=True
transformer.h.0.attn.c_proj.weight: 589824 parameters, requires_grad=True
transformer.h.0.attn.c_proj.bias: 768 parameters, requires_grad=True
transformer.h.0.ln_2.weight: 768 parameters, requires_grad=True
transformer.h.0.ln_2.bias: 768 parameters, requires_grad=True
transformer.h.0.mlp.c_fc.weight: 2359296 parameters, requires_grad=True
transformer.h.0.mlp.c_fc.bias: 3072 parameters, requires_grad=True
transformer.h.0.mlp.c_proj.weight: 2359296 parameters, requires_grad=True
transformer.h.0.mlp.c_proj.bias: 768 parameters, requires_grad=True
transformer.h.1.ln_1.weight: 768 parameters, requires

### Train the model

In [19]:
# Use torchmetrics to set up accuracy, recall, precision, and auroc
accuracy = Accuracy(task = 'binary').to(device)
recall = Recall(task = 'binary').to(device)
precision = Precision(task = 'binary').to(device)
auroc = AUROC(task = 'binary').to(device)

In [20]:
# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = [element.to(device) for element in batch]

        # Unpack the batch
        token_id, attention_mask, label = batch

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(token_id, attention_mask=attention_mask, labels=label)
        loss = train_output.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += token_id.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = [element.to(device) for element in batch]

        # Unpack the batch
        token_id, attention_mask, label = batch

        with torch.no_grad():
          # Forward pass
          eval_output = model(token_id, attention_mask=attention_mask, labels=label)

        # Calculate validation metrics
        labels = label
        predicted_labels = torch.argmax(eval_output.logits, axis=1)

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(predicted_labels, labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch:  50%|█████     | 1/2 [00:40<00:40, 40.43s/it]


	 - Train loss: 0.1153
	 - Validation Accuracy: 0.9902
	 - Validation Precision: 0.8657
	 - Validation Recall: 0.8352
	 - Validation AUROC: 0.8735



Epoch: 100%|██████████| 2/2 [01:18<00:00, 39.21s/it]


	 - Train loss: 0.0287
	 - Validation Accuracy: 0.9911
	 - Validation Precision: 0.8674
	 - Validation Recall: 0.8667
	 - Validation AUROC: 0.8877



### Test on a specific sentence, see the outcome

In [21]:
new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
    output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'Spam' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'Ham'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Predicted Class:  Spam


### Questions

Question 1: Run the above by fine tuning GPT2 and the classfier head and by not doing this (using GPT2 as a feature encoder). What is the gap between this? What are the metrics we get in each case?

Solution: FILL Paste your output here

In [22]:
# Load the BertForSequenceClassification model
# Do not ouput the attentions and all hidden states

config = GPT2Config.from_pretrained(
    'gpt2',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)

# Set to 'gpt2' (the smallest GPT2 which is 120 M parameters)
# Use the config above and set other labels as needed
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=config)

# Set the pad token id to the eos token id
model.config.pad_token_id = model.config.eos_token_id

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5
# See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-08
)

model = model.to(device)

# Recommended number of epochs: See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

# Print all the layers of this GPT2 model and the number of parameters per layer
# If this is False, fine tune just the classifier layer and leave all other GPT2 parameters alone
# If this is True, fine tune everything
fine_tune = False

# Turn off gradients using the above
total_parameters = 0
for layer, param in model.named_parameters():
  # num = param.numel()
  # total_parameters += num
  # print(layer, num, param.shape, fine_tune)
  param.requires_grad = False
  if not fine_tune:
    if 'score' in layer or 'classifier' in layer:
      param.requires_grad = True

# Main training / validation loop
for _ in trange(epochs, desc = 'Epoch'):

    # ========== Training ==========

    # Set model to training mode
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        # Put each element of batch onto the device
        batch = [element.to(device) for element in batch]

        # Unpack the batch
        token_id, attention_mask, label = batch

        # Set gradients to zero
        optimizer.zero_grad()

        # Forward pass
        train_output = model(token_id, attention_mask=attention_mask, labels=label)
        loss = train_output.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += token_id.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_auroc = []

    for batch in validation_dataloader:
        batch = [element.to(device) for element in batch]

        # Unpack the batch
        token_id, attention_mask, label = batch

        with torch.no_grad():
          # Forward pass
          eval_output = model(token_id, attention_mask=attention_mask, labels=label)

        # Calculate validation metrics
        labels = label
        predicted_labels = torch.argmax(eval_output.logits, axis=1)

        val_accuracy.append(accuracy(predicted_labels, labels))
        val_recall.append(recall(predicted_labels, labels))
        val_precision.append(precision(predicted_labels, labels))
        val_auroc.append(auroc(predicted_labels, labels))

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)))
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)))
    print('\t - Validation AUROC: {:.4f}\n'.format(sum(val_auroc)/len(val_auroc)))


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
Epoch:  50%|█████     | 1/2 [00:11<00:11, 11.51s/it]


	 - Train loss: 0.6158
	 - Validation Accuracy: 0.8586
	 - Validation Precision: 0.1286
	 - Validation Recall: 0.0617
	 - Validation AUROC: 0.4775



Epoch: 100%|██████████| 2/2 [00:22<00:00, 11.49s/it]


	 - Train loss: 0.3667
	 - Validation Accuracy: 0.9028
	 - Validation Precision: 0.5048
	 - Validation Recall: 0.3364
	 - Validation AUROC: 0.6171



Fine-tuning the entire GPT-2 model along with the classifier head results in significantly higher accuracy, precision, recall, and AUROC compared to using GPT-2 as a feature encoder.

The gap in performance between the two approaches is substantial, indicating that fine-tuning the model for the specific downstream task is effective.
Fine-tuning improves both classification accuracy and the model's ability to capture relevant features for the task.

In contrast, using GPT-2 as a feature encoder without fine-tuning results in lower performance, as the model's pre-trained features might not be tailored to the specific task.

Therefore, based on these results, fine-tuning GPT-2 is recommended for achieving better performance on the given classification task.